# Spark DataFrame Basics

In [2]:
pip install pyspark

  Using cached pyspark-3.4.1.tar.gz (310.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285432 sha256=3f0ce4247b3e3a690aad77de03805139e3cb49c7de182bb87659405e8f519b3e
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\e9\b4\d8\38accc42606f6675165423e9f0236f8e825f6b6b6048d6743e
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyspark[sql,ml,mllib]

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
# json 데이터 읽기
df = spark.read.json('people.json')

In [4]:
# 데이터 보기
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
# 데이터프레임의 스키마 확인
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
# 데이터프레임 열만 확인
df.columns # pandas와 같다

['age', 'name']

In [7]:
# 데이터프레임의 통계적 요약(null값은 포함시키지 않음)
df.describe()
# DataFrame[summary: string, age: string, name: string]이 반환된다.
# 문자열을 지닌 summary, age, name이라는 열을 불러왔다는 의미이다.
# (출력값을 확인해보니 age, summay가 문자라는 의미인듯)

DataFrame[summary: string, age: string, name: string]

In [8]:
# 값을 확인하려면 .show()
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [9]:
# 좋은 데이터나 특정 소스의 데이터가 아니라면 보통 스키마를 명확히 해야한다.
# 어떤 열이 문자열이고 정수인지 등등을 스키마가 정확히 알고 있어야 한다.
# 가장 먼저 타입 툴을 불러와야 한다.
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

# 다음 단계로는 구조 필드의 리스트를 만들어줘야한다.
# StructField의 매개 변수는 이름, 데이터 유형, 널 값이 될 수 있는지 여부 변수까지 3개가 있다.
data_schema = [StructField('age',IntegerType(),True),
# 이 필드와 관련된 열은 '나이'열이며, 나이는 정수로 표현되며, 널값을 가질 수 있다는 의미이다.
# True가 아닌 False를 사용하면 null값을 넣었을 때 에러가 발생한다.
              StructField('Name',StringType(),True)]

In [10]:
# 최종 구조로 final_struc 변수를 설정한다
# fields에는 우리가 지정한 스키마를 대입한다.
final_struc = StructType(fields=data_schema)

In [11]:
# 위 작업을 실행시키고, json 파일을 불러올 예정인데 스키마가 내가 원하는 최종 구조임을 명시한다.
# 스키마를 매개 변수 인자로  넣어 최종 구조를 불러온다.
df = spark.read.json('people.json', schema=final_struc)

In [13]:
df.printSchema()
# 코드를 실행시키면 우리가 정의한 스키마대로 출력되는것을 볼 수 있다.
# (age의 데이터유형이 long에서 integer로 바뀐것을 확인할 수 있다.)
# 만약에 특정 파일 포맷이 문자열로 읽히게 디폴트 설정이 되어 있다면
# 이 방법을 통해 스키마 유형을 특정지을 수 있다.
# 특히 난해한 데이터 유형을 사용할 때 힘든데 다행히도 스파크가 유형을 추론하는
# 능력이 좋아지고 있다.
# 우리 강의 자료는 csv파일이여서 스파크가 스키마 유추를 하는데 문제가 없다.

root
 |-- age: integer (nullable = true)
 |-- Name: string (nullable = true)



# Spark DataFrame Basic2

In [14]:
# json 데이터 읽기
df = spark.read.json('people.json')

In [15]:
df['age']

Column<'age'>

In [16]:
type(df['age'])

pyspark.sql.column.Column

In [17]:
df.head(2)
# [Row(age=None, name='Michael'), Row(age=30, name='Andy')]
# 각 행으로 이루어진 리스트 반환 -> index사용 가능

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [18]:
df.head(2)[0]

Row(age=None, name='Michael')

In [19]:
type(df.head(2)[0])

pyspark.sql.types.Row

In [20]:
# 열 선택
df.select('age')
# 반환 타입이 데이터프레임이다.

DataFrame[age: bigint]

In [21]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [22]:
# 여러 열 선택
df.select(['age','name'])

DataFrame[age: bigint, name: string]

In [23]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [24]:
# 새 열 만들기
# withColumn : 기본적으로 새 열을 추가하거나, 기존 열을 대체해서 새 데이터 집합을 반환
df.withColumn('newage', df['age']).show()
# 'newage' 컬럼이 추가된것을 볼 수 있다.
# 'age'컬럼이 대체되는 것이 아님

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [28]:
# 기존 컬럼값을 대체할 수 있다.
df.withColumn('age', df['name']).show()

+-------+-------+
|    age|   name|
+-------+-------+
|Michael|Michael|
|   Andy|   Andy|
| Justin| Justin|
+-------+-------+



In [29]:
# 연산을 적용할 수도 있다.
df.withColumn('double_age', df['age']*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [30]:
# 기억해야할건, 이 변화들이 원조 데이터 프레임에서 영구적이지 않다.
df.show()
# 원조 데이터프레임에는 변화가 없다.

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [31]:
# 컬럼 이름 변경
df.withColumnRenamed('age','my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [32]:
# pure sql에 대하여..
# 직통으로 데이터프레임을 가지고 상호작용하는 쿼리 언어
# 데이터프레임을 sql temporary view로 등록해야한다.
# createOrReplaceTempView: view 생성. 이미 view가 존재하면 replace한다.
df.createOrReplaceTempView('people')
# people'은 view 이름을 설정하는듯하다.

In [33]:
# 사용 방법
results = spark.sql('SELECT * FROM people')
results.show()
# 출력: df 테이블이 출력된다.

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [34]:
new_results = spark.sql('SELECT * FROM people WHERE age=30')
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [25]:
# 장점: 몇 가지 기본 스파트 데이터프레임 작업을 잊어도 복잡한 작업을 빨리 할 수 있다.
# 하지만 본 과정에서는 중점적으로 사용하지 않는다.
# python과 pyspark의 특정 방법에 집중한다.
# 하지만 sql 지식이 있다면 정말 유용한 도구가 된다.